<a href="https://colab.research.google.com/github/nerdk312/Model-based-RL/blob/master/Simplified_Model_based_Pacman_ATARI_labels_120420.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

# Installation and Imports

In [0]:
!pip install git+git://github.com/mila-iqia/atari-representation-learning.git
!pip install git+git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail
!pip install git+git://github.com/openai/baselines
!pip install wandb

  Cloning git://github.com/mila-iqia/atari-representation-learning.git to /tmp/pip-req-build-ipb6kck2
  Running command git clone -q git://github.com/mila-iqia/atari-representation-learning.git /tmp/pip-req-build-ipb6kck2
  Created wheel for atariari: filename=atariari-0.0.1-cp36-none-any.whl size=46584 sha256=a9cc4877d2903fa6f0aecd05632ba6586a3778cdf9ac22e59a3a72c4862651e8
  Stored in directory: /tmp/pip-ephem-wheel-cache-se83lk29/wheels/3d/69/51/5e436e5ae566c5b4dec5c53e65396d516459877a42a11d7aa4
Successfully built atariari
  Cloning git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail to /tmp/pip-req-build-xwlmfwkg
  Running command git clone -q git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail /tmp/pip-req-build-xwlmfwkg
  Created wheel for a2c-ppo-acktr: filename=a2c_ppo_acktr-0.0.1-cp36-none-any.whl size=18833 sha256=6975a1f6ec5973f57824b7dc3d9cdfb6e67b79d0c816e240aabe8a825ee6c55f
  Stored in directory: /tmp/pip-ephem-wheel-cache-nvdbzz79/wheels/91/52/02/ec5c530fd76d56a66

In [0]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os

from __future__ import print_function
import pickle
import sys
sys.path.append('/content/gdrive/My Drive/Unsupervised_state_representation/atariari')

import wandb

import argparse

import random
import torch
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import RandomSampler, BatchSampler
from torchvision import transforms
import torchvision.transforms.functional as TF

from tqdm import tqdm
import datetime
import time

from atariari.benchmark.envs import *
from torch.autograd import Variable

import gym
from atariari.benchmark.wrapper import AtariARIWrapper

#from benchmark import *
#from methods import utils

# Needed to create dataloaders
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Imported required for the Model-based RL
from sklearn.preprocessing import StandardScaler

In [0]:
!wandb login ##############

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


# Model setup

In [0]:
class EarlyStopping_loss(object):
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=5, verbose=False, wandb=None, name=""):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0 #  Counter which checks for early stopping
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = 1e11 # Nawid - Set a very high initial best loss
        self.name = name
        self.wandb = wandb
        self.epoch_counter = 0 # Counter which counts the epochs
        self.checkpoint_counter = 0 # Counter which saves after 10 successful decreases in value

    def __call__(self, val_loss, model,optimizer):

        score = val_loss
        self.epoch_counter +=1

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model,optimizer)
        elif score >= self.best_score: # Nawid - Inverse signs to take into minimising loss instead of maximising accuracy
            self.counter += 1
            print(f'EarlyStopping for {self.name} counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                print(f'{self.name} has stopped')
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model,optimizer)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, optimizer):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(
                f'Validation loss decreased/improved for {self.name}  ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        
        current_checkpoint = {
            'epoch': self.epoch_counter,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_loss,
            'lr':optimizer.state_dict()['param_groups'][0]['lr']} # Saves the epoch number, model state dict, optimizer state, loss and learning rate
        

        save_dir = self.wandb.run.dir
        torch.save(current_checkpoint, save_dir + "/" + self.name + ".pt")
        self.wandb.save(save_dir + "/" + self.name + ".pt")
        #torch.save(model.state_dict(), save_dir + "/" + 'Epoch_no_{}_'.format(self.epoch_counter)+self.name + ".pt")
        #self.wandb.save(save_dir + "/" + self.name + ".pt")
        self.val_loss_min = val_loss

        self.checkpoint_counter += 1

        if self.checkpoint_counter % 10 == 0: # Saves the optimiser and the state model
            checkpoint = {
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict' : optimizer.state_dict(),
              'loss': val_loss,
              'lr': optimizer.state_dict()['param_groups'][0]['lr']}
              
            torch.save(checkpoint, save_dir + "/" + 'Epoch_no_{}_'.format(self.epoch_counter)+self.name + "_checkpoint.pt") 
            self.wandb.save(save_dir + "/" + 'Epoch_no_{}_'.format(self.epoch_counter)+self.name + "_checkpoint.pt")

In [0]:
class NNDynamicsModel(nn.Module):
    '''
    Model that predict the next state, given the current state and action
    '''
    def __init__(self, input_dim, obs_output_dim):
        super(NNDynamicsModel, self).__init__()

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(num_features=512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(256, obs_output_dim)
        )
    def forward(self, x):
        return self.mlp(x.float())

class NNRewardModel(nn.Module):
    '''
    Model that predict the reward given the current state and action
    '''
    def __init__(self, input_dim, reward_output_dim):
        super(NNRewardModel, self).__init__()

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(num_features=512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(256, reward_output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.mlp(x.float())

def model_MSEloss(y_pred,y_truth, device):
    '''
    Compute the MSE (Mean Squared Error)
    '''
    y_truth = torch.FloatTensor(np.array(y_truth)).to(device)
    return F.mse_loss(y_pred.view(-1).float(), y_truth.view(-1))

In [0]:
def train_model(train_data,val_data,model,batch_size, max_model_iter, optimizer, device,early_stopper,desired_model='Env_model'):
    ''' 
    General function to train either of the two models
    '''
    # Unpack data
    (X_train, y_train), (X_val, y_val) =  train_data, val_data
    losses_env = []

    loss_function = model_MSEloss

    # go through max_model iter supervised iterations
    for it in tqdm(range(max_model_iter)):
        # create mini batches of size batch_size
        for mb in range(0,len(X_train), batch_size): # Nawid- Batch size is the step size
            if len(X_train) > mb + BATCH_SIZE:
                X_mb = X_train[mb:mb+BATCH_SIZE]
                y_mb = y_train[mb:mb+BATCH_SIZE]
                #X_mb += np.random.normal(loc = 0, scale = 0.001, size= X_mb.shape)

                optimizer.zero_grad()
                # forward pass of model to compute the output
                pred_mb = model(torch.tensor(X_mb).to(device))
                
                # compute the loss
                loss = loss_function(pred_mb,y_mb,device) #Nawid-  Uses Mse loss if dynamics model or uses BCE loss if reward/done model
                wandb.log({'{} Training loss'.format(desired_model):loss.cpu().detach().numpy()})
                # backward pass
                loss.backward()
                # optimization step
                optimizer.step()
        
        # Nawid - Calculate the validation loss after each epoch
        model.eval()
        with torch.no_grad():
            pred_val = model(torch.tensor(X_val).to(device))
            val_loss = loss_function(pred_val, y_val,device)
            wandb.log({'{} Validation loss'.format(desired_model):val_loss})
        
        # Checks whether to early stop after each epoch
        
        early_stopper(val_loss,model,optimizer)
        if early_stopper.early_stop:
            print("Early stopping")
            break

        print('epoch: {} completed'.format(it))

# General Functions

In [0]:
class General_functions():
    def __init__(self, ENV_NAME, feature_size,n_actions,possible_positions,state_mode = 'ATARIARI',encoder=None):
        self.state_mode = state_mode
        self.env = AtariARIWrapper(gym.make(ENV_NAME))
        self.initial_info_labels = self.env.labels()

        self.feature_size = feature_size 

        self.repeated_initial = True # Nawid - set repeated initial as true initially
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.encoder= encoder
        self.n_actions = n_actions
        # possible positions is numpy array with most of the possible positions that the agent can go to
        self.possible_positions = possible_positions
        self.possible_positions_list = self.possible_positions.tolist()

    def one_hot(self,i):
        a = np.zeros(self.n_actions, 'uint8')
        a[i] = 1
        return a

    def state_conversion(self,obs,info_labels):
        if self.state_mode == 'ATARIARI':
            state = []
            i = 0             
            for key in info_labels:
                if i < self.feature_size: # Nawid - Only the first 14 info are crucial I believe
                    state.append(info_labels[key])
                    i +=1
                else:
                    state = np.array(state)
                    return state
                    
                '''
                if key == 'player_x' or key == 'player_y':
                    state.append(info_labels[key])
                
            state = np.array(state)
            return state
                '''
                #print(counter,key)
    
    def next_position(self,state, action):
        #print('state', state, state.shape)

        next_position = state[8:]
        #next_position = np.reshape(next_position, (1,2))
        if action == 0:
            pass
        elif action == 1:
            next_position[1] = next_position[1] - 2 
        elif action == 2:
            next_position[0] = next_position[0] + 2
        elif action == 3:
            next_position[0] = next_position[0] - 2
        elif action == 4:
            next_position[1] = next_position[1] + 2
        
        #print('next position',next_position,next_position.shape)
        if next_position.tolist() in self.possible_positions_list: # possible positions will be a list which is fed into the network 
            #print('present')
            return True
        else:
            #print('not present')
            return False

        
    def random_action_selection(self, state):
        while True:
            action = np.random.choice(5, 1, p=[0.04, 0.24, 0.24, 0.24, 0.24]).squeeze() # Samples a random action with these given probabilites
            feasible_action = self.next_position(state,action)
            if feasible_action:
                #print('feasible action')
                return action
    
            
    def check_initial(self, next_info_labels):
        # Checks if there has been a change from the initial info labels to show that the initial lag period is over 
        if self.initial_info_labels == next_info_labels:
            pass
        else:
            self.repeated_initial = False

    def reward_conversion(self,reward,done,info_labels,next_info_labels):
        # checks whether there has been a change in lives- change done to 1, otherwise the done should be fine regardless
        if not info_labels['num_lives'] == next_info_labels['num_lives']: # Nawid - If there is a change in the number of lives -  The change in the number of lives occurs after the mspacman is resurrected
            done = True 
            self.repeated_initial = True
            self.initial_info_labels  = next_info_labels # Sets the new initial labels.
        return reward, done  

# Data collection and preprocessing

In [0]:
class Data_collection(General_functions):
    def __init__(self,ENV_NAME, feature_size,n_actions, possible_positions, state_model = 'ATARIARI', encoder= None):
        super(Data_collection,self).__init__(ENV_NAME,feature_size,n_actions,possible_positions, state_model, encoder)
        
    def gather_random_trajectories(self,num_traj):
        dataset_random = []
        #Env name could either be the RAM case or the generic case
        # set pacman_done as initially zero
        pacman_done = 0
        i = 0
        for n in range(num_traj):
            if n % 10 ==0:
                print('trajectory number :',n)
            # Initial set up
            obs = self.env.reset()
            self.env.seed(0)

            self.repeated_initial = True # Nawid- Used to represent the initial state
            info_labels = self.env.labels() # Nawid -  Used to get the current state
            state = self.state_conversion(obs, info_labels) # Used to get the initial state
            while True:
                # Choosing action and env step         
                #sampled_action = np.random.randint(0,n_actions)
                sampled_action = self.random_action_selection(state)

                sampled_action_one_hot = self.one_hot(sampled_action)
                next_obs, reward, done, next_info = self.env.step(sampled_action)
                next_info_labels = next_info['labels']
                #print(next_info_labels)
                # self.repeated initial is set to true at first and it is is turned to true in the reward_collection class when done is true
                
                self.check_initial(next_info_labels) # checks if the initial value and the current value is the same, if it is not then it changes the repeated_initial bool
                if self.repeated_initial: # If the initial state is repeating
                    pass
                        
                else:
                    # New state achieved, so save data
                    state = self.state_conversion(obs, info_labels)
                    next_state = self.state_conversion(next_obs, next_info_labels)
                    # looks if ep is done and sets the new initial info state and sets the repeated_initial bool back to true
                    reward,pacman_done = self.reward_conversion(reward, done, info_labels, next_info_labels) 
                    dataset_random.append([state, next_state,reward,pacman_done,sampled_action_one_hot])
                    
                obs = next_obs
                info_labels = next_info_labels

                # ends the loop when its done or pacman_done in the single life setting
                if done or (single_life and pacman_done):
                    break 
        return dataset_random

    def collate_data(self,random_dataset, rl_dataset):
        rand_data = np.array(random_dataset)
        num_rand_examples = len(rand_data)
        D_train = rand_data[:int(-num_rand_examples*1/5)] 
        D_valid = rand_data[int(-num_rand_examples*1/5):]
        print("number random examples:",num_rand_examples, 'len(D_train_rand)', len(D_train),'len(D_valid_rand)', len(D_valid))
        if len(rl_dataset) > 0:
            # Adds the rl dataset to the random dataset if there is any present
            rl_data = np.array(rl_dataset)
            num_rl_examples = len(rl_data)
            D_rl_train = rl_data[:int(-num_rl_examples*1/5)] 
            D_rl_valid = rl_data[int(-num_rl_examples*1/5):]
                        
            D_train = np.concatenate([D_train, D_rl_train], axis = 0)
            D_valid = np.concatenate([D_valid, D_rl_valid], axis = 0)
            print("number rl examples:",num_rl_examples, 'len(D_rl_train)', len(D_rl_train),'len(D_valid_rand)', len(D_rl_valid))
            
        #print("len(D_train):", len(D_train), 'len(D_valid)', len(D_valid))

        # Shuffle the dataset
        
        sff = np.arange(len(D_train))
        np.random.shuffle(sff)
        D_train = D_train[sff]
        #print('D_train shape',D_train.shape)


        # Create the input and output for the train
        X_train_obs = np.array([obs for obs,_,_,_,_ in D_train]) # Takes obs and action
        X_train_obs = X_train_obs.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)
        X_train_act = np.array([act for _,_,_,_,act in D_train])
        

        # Env output
        y_env_train = np.array([no for _,no,_,_,_ in D_train])
        y_env_train = y_env_train.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)
        y_env_train = y_env_train - X_train_obs 
        #y_env_train = y_env_train - np.array([obs for obs,_,_,_,_ in D_train]) # y(state) = s(t+1) - s(t)

        
        # Next state output
        X_val_obs = np.array([obs for obs,_,_,_,_ in D_valid]) # Takes obs and action
        X_val_obs = X_val_obs.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)        
        X_val_act = np.array([act for _,_,_,_,act in D_valid])

        y_env_val = np.array([no for _,no,_,_,_ in D_valid])
        y_env_val = y_env_val.astype(np.int16)
        y_env_val = y_env_val - X_val_obs 
        #y_env_val = y_env_val - np.array([obs for obs,_,_,_,_ in D_valid]) # y(state) = s(t+1) - s(t)

        
    
        env_train_data, env_val_data = (X_train_obs, X_train_act, y_env_train), (X_val_obs, X_val_act, y_env_val)
        

        return env_train_data, env_val_data 

    def setup_dataset(self,env_train_data, env_val_data):
        
        # Unpack data
        (X_env_train_obs, X_env_train_act, y_env_train), (X_env_val_obs, X_env_val_act, y_env_val) = env_train_data, env_val_data
    
        # Concatentates the normalised states with the one hot vector for the actions
        X_env_train = np.concatenate((X_env_train_obs,X_env_train_act),axis=1)
        X_env_val = np.concatenate((X_env_val_obs,X_env_val_act),axis=1)

        # Pack data tuples
        env_train_data, env_val_data = (X_env_train, y_env_train),(X_env_val, y_env_val) 


        return env_train_data, env_val_data
    
'''
data_object = Data_collection(ENV_NAME,feature_size,n_actions,possible_positions)
data = data_object.gather_random_trajectories(10)
print(data[0])
rl_dataset = []
env_train, env_val = data_object.collate_data(data,rl_dataset)
print(env_train)
norm_env_train, norm_env_val, = data_object.setup_dataset(env_train, env_val)
'''

'\ndata_object = Data_collection(ENV_NAME,feature_size,n_actions,possible_positions)\ndata = data_object.gather_random_trajectories(10)\nprint(data[0])\nrl_dataset = []\nenv_train, env_val = data_object.collate_data(data,rl_dataset)\nprint(env_train)\nnorm_env_train, norm_env_val, = data_object.setup_dataset(env_train, env_val)\n'

# Controller

In [0]:
class multi_model_based_control(General_functions):
    def __init__(self,ENV_NAME, feature_size,n_actions,possible_positions,env_model,rew_model, num_sequences,horizon_length, state_model = 'ATARIARI', encoder= None):
        super(multi_model_based_control,self).__init__(ENV_NAME,feature_size,n_actions,possible_positions, state_model, encoder)
        
        self.env_model = env_model
        self.rew_model = rew_model
        self.horizon_length = horizon_length
        self.num_sequences = num_sequences        
        self.n_actions = n_actions
        self.global_step = 0
        self.rl_dataset = []

    def random_sampling_shooting(self,real_obs):
        '''
        Use a random-sampling shooting method, generating random action sequences. The first action with the highest reward of the entire sequence is returned
        '''
        best_reward = -1e9
        best_next_action = []
        m_obs = np.array([real_obs for _ in range(self.num_sequences)])
        

        # array that contains the rewards for all the sequence
        unroll_rewards = np.zeros((self.num_sequences, 1)) 
        first_sampled_actions = []

        self.env_model.eval()
        self.rew_model.eval()

        # Create a batch of size 'num_sequences' (number of trajectories) to roll the models 'horizon length' times
        ## i.e roll a given number of trajectories in a single batch (to increase speed)
        for t in range(self.horizon_length):
            # sample actions for each sequence
            sampled_actions = np.array([np.random.randint(0,self.n_actions) for _ in range(self.num_sequences)])
            #print('sampled actions', sampled_actions.shape)
            #print(sampled_actions[0])
            sampled_actions_one_hot = np.array([self.one_hot(action) for action in sampled_actions])
            #print('sampled actions one hot',sampled_actions_one_hot.shape)
            #print(sampled_actions_one_hot[0])

        
            env_model_input = np.concatenate([m_obs, sampled_actions_one_hot], axis = 1)            
            #print('env model input shape',env_model_input.shape)
            # compute the next state for each sequence
            #print('m obs',m_obs[0])
            pred_obs = self.env_model(torch.tensor(env_model_input).to(self.device))
            #print('pred obs',pred_obs.shape)
            #print('pred_obs',pred_obs[0])

            # add previous observation
            next_obs = pred_obs.cpu().detach().numpy() + m_obs
            #print('next_obs',next_obs.shape)
            #print('next_obs',next_obs[0])


            pred_rew = self.rew_model.predict_reward(next_obs)
            #print('pred rew',pred_rew.shape)
            #print('unroll rewards', unroll_rewards.shape)
            unroll_rewards += pred_rew
        
            m_obs = next_obs # Nawid - Update the state after calculating the new state and calculating the rewards

            if t ==0:
                first_sampled_actions = sampled_actions
                #print('first sampled actions',first_sampled_actions.shape) 
        self.env_model.train()
        self.rew_model.train()

        # Best the position of the sequence with the higher reward
        
        arg_best_reward = np.argmax(unroll_rewards)
        #print('arg_best reward', arg_best_reward)
        #print('best reward',unroll_rewards[arg_best_reward])

        #print('differet reward',unroll_rewards[arg_best_reward - 1])

        best_sum_reward = unroll_rewards[arg_best_reward].squeeze()
        # take the first action of this sequence
        best_action = first_sampled_actions[arg_best_reward]
        #best_action =  np.squeeze(best_action)
        return best_action, best_sum_reward
    
    def on_policy_collection(self,STEPS_PER_AGGR):
        obs = self.env.reset()
        self.env.seed(0) # Set the random seed of the environment

        num_examples_added = 0
        game_reward = 0
        # records how long the agent survives for
        timesteps = 0
        controller_pred_rews = []
        rews = []
        # records how many simulations have occurred

        while num_examples_added < STEPS_PER_AGGR:
            while True:
                tt = time.time()
                if self.repeated_initial: # Placed the repeated initial before the check_initial info labels since if the actions were chosen by MPC for the 264 intial actions, it would increase the computational time by quite a lot
                    obs,_,_,info = self.env.step(0) # Any action taken
                    info_labels = info['labels']
                    # Checks if the initial set is being repeated and if it isnt, it sets the value off
                    self.check_initial(info_labels) 
                else:
                    # new state achieved
                    state = self.state_conversion(obs,info_labels)
                    action, pred_rew = self.random_sampling_shooting(state)
                    action_one_hot = self.one_hot(action)
                    controller_pred_rews.append(pred_rew)

                    # one step in the environment with the action returned by
                    next_obs, reward, done, next_info = self.env.step(action)
                    next_info_labels = next_info['labels']
                
                    next_state = self.state_conversion(next_obs, next_info_labels)
                    # Obtains the reward, done and sets whether repeated initial should be true or not
                    reward, pacman_done = self.reward_conversion(reward,done,info_labels, next_info_labels) 

                    # add to the RL dataset                
                    self.rl_dataset.append([state, next_state, reward, pacman_done, action_one_hot])

                    num_examples_added += 1
                    timesteps +=1 
                    game_reward += reward
                    obs = next_obs
                    info_labels = next_info_labels
                                 
                    if done or (single_life and pacman_done):
                        self.global_step += 1
                        obs = self.env.reset()
                        self.env.seed(0) # Need to set the random seed after the environment is done
                        wandb.log({'pred_rew': np.mean(controller_pred_rews)*100,'survival time':timesteps,'global step':self.global_step }) # scaled the prediction by 100 to make it visible for comparison on the plot 
                        print('  >> R: {:.2f}, Mean sum:{:.2f},Survival time: {},Num examples:{}'.format(game_reward, np.mean(controller_pred_rews),timesteps,num_examples_added))
                        rews.append(game_reward)
                        game_reward = 0
                        timesteps = 0
                        controller_pred_rews = []
                        break

        print('  >> Mean: {:.2f}', np.mean(rews)) 
        return self.rl_dataset   

# Oracle

In [0]:
class Reward_Oracle():
    
    def predict_reward(self, next_state):
        #print('next_state',next_state[:,0])

        agent_position = np.transpose(np.array((next_state[:,8], next_state[:,9])))
        #print('agent position', agent_position.shape)

        enemy1_position = np.transpose(np.array((next_state[:,0], next_state[:,4])))
        #print('enemy_position1',enemy1_position.shape)
        enemy2_position = np.transpose(np.array((next_state[:,1], next_state[:,5])))
        #print('enemy_position2',enemy2_position.shape)
        enemy3_position = np.transpose(np.array((next_state[:,2], next_state[:,6])))
        #print('enemy_position3',enemy3_position.shape)
        enemy4_position = np.transpose(np.array((next_state[:,3], next_state[:,7])))
        #print('enemy_position4',enemy4_position.shape)

        distance_1 = np.array([np.linalg.norm(agent_position-enemy1_position, axis=1)])
        distance_2 = np.array([np.linalg.norm(agent_position-enemy2_position, axis=1)])
        distance_3 = np.array([np.linalg.norm(agent_position-enemy3_position, axis=1)])
        distance_4 = np.array([np.linalg.norm(agent_position-enemy4_position, axis=1)])
        overall_dist = np.transpose(np.concatenate((distance_1,distance_2, distance_3, distance_4),axis = 0))
        #print('overall dist',overall_dist.shape)
        min_dist = np.amin(overall_dist, axis = 1)

        #print('min dist',min_dist.shape)
        min_dist[min_dist<=threshold_distance] = 0
        min_dist[min_dist>threshold_distance] = 1
        #min_dist[min_dist<10] = 0
        #min_dist[min_dist>10] = 1
        min_dist = np.expand_dims(min_dist, axis = 1)
        return min_dist

    def eval(self):
        pass

    def train(self):
        pass    

# Hyperparameters

In [0]:
wandb.init(entity="nerdk312", project="ATARI_LABELS_MPC")
ENV_NAME = 'MsPacman-ramDeterministic-v4' #'MsPacmanNoFrameskip-v4'  #'MsPacmanDeterministic-v4' # 'MsPacmanNoFrameskip-v4'
feature_size = 10 # only predict the agent state 10 

# Main loop hyperp
env_model_pretrain = False
pretrained_env_model = '/content/Env_model_17_4_2020-10_32.pt' #'/content/Epoch_no_43_Env_model_13_4_2020-8_22_checkpoint.pt'

ENV_LEARNING_RATE = 1e-4
rew_oracle = True

AGGR_ITER = 100
STEPS_PER_AGGR = 30000

# Random MB hyperp
NUM_RAND_TRAJECTORIES = 3000

# 'cuda' or 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# Supervised Model Hyperp
BATCH_SIZE = 1024
TRAIN_ITER_MODEL =  100

# Controller Hyperp
HORIZON_LENGTH = 5
NUM_ACTIONS_SEQUENCES = 200

load_data = True
if load_data:
    loaded_trajectories = '/content/gdrive/My Drive/MsPacman-data/Random trajectories/No_wall_movements/rand_traj_3000-_all_lives_16_4_2020-14_33.npy'#'/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-16_4_2020-14_33.npy'#'/content/gdrive/My Drive/MsPacman-data/rand_traj_10000-16_4_2020-12_14.npy' #'/content/gdrive/My Drive/MsPacman-data/rand_traj_5000-15_4_2020-12_9.npy' #'/content/gdrive/My Drive/MsPacman-data/rand_traj_3000-11_4_2020-11_21.npy'

collect_data = False

observation_channels = 1
action_dim = 1
n_actions = 5 #9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.
reward_dim = 1

# Decides if an episode is done or not
single_life = False

# Time and date information
now = datetime.datetime.now()
date_time = "{}_{}_{}-{}_{}".format(now.day,now.month,now.year, now.hour, now.minute)


possible_positions = np.load('/content/gdrive/My Drive/MsPacman-data/possible_pacman_positions.npy',allow_pickle=True)
# Making the network deterministic - https://pytorch.org/docs/stable/notes/randomness.html
random_seed = 0

threshold_distance = 5 

normalise_on = False
if random_seed:    
    torch.manual_seed(1)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    #np.random.seed(0)



config = wandb.config
config.batch_size = BATCH_SIZE          
config.horizon_length = HORIZON_LENGTH
config.num_action_seq = NUM_ACTIONS_SEQUENCES
config.num_rand_trajectories = NUM_RAND_TRAJECTORIES
config.aggr_iter = AGGR_ITER
config.steps_per_aggr = STEPS_PER_AGGR

config.train_model_iter = TRAIN_ITER_MODEL
config.env_lr = ENV_LEARNING_RATE
config.env_model_pretrain = env_model_pretrain
config.pretrained_env =  pretrained_env_model

config.rew_oracle = rew_oracle
     
config.no_actions = n_actions
config.load_data = load_data
config.collect_data = collect_data
config.single_life = single_life
config.random_seed = random_seed

config.threshold_distance = threshold_distance
if load_data:
    config.loaded_trajectories = loaded_trajectories

cuda


# Main - Model training

Data collection and processing

In [0]:
# gather the dataset of random sequences
data_collector = Data_collection(ENV_NAME,feature_size,n_actions,possible_positions)
if load_data:
    rand_dataset = np.load(loaded_trajectories,allow_pickle=True)
else:
    rand_dataset = data_collector.gather_random_trajectories(NUM_RAND_TRAJECTORIES)
    filename = '/content/gdrive/My Drive/MsPacman-data/rand_traj_{}-'.format(NUM_RAND_TRAJECTORIES) + date_time
    np.save(filename,rand_dataset)

#rand_dataset = np.array([x for x in rand_dataset if not same(x)])
#rand_dataset = rand_dataset[0:100,:]
empty_dataset = []
env_train_data, env_val_data = data_collector.collate_data(rand_dataset,empty_dataset)

norm_env_train_data, norm_env_val_data = data_collector.setup_dataset(env_train_data, env_val_data)

number random examples: 1290118 len(D_train_rand) 1032095 len(D_valid_rand) 258023


Dynamics model training

In [0]:

env_model = NNDynamicsModel(n_actions + feature_size, feature_size).to(device) # Nawid - Need to put both actions as it is a one-hot vector


if env_model_pretrain:
    print('checkpoint')
    #env_model.load_state_dict(torch.load(pretrained_env_model))
    checkpoint = torch.load(pretrained_env_model)
    env_model.load_state_dict(checkpoint['model_state_dict'])
    
    for parameter in env_model.parameters():
        parameter.requires_grad = False
    env_model.eval()

else:
    env_optimizer = torch.optim.Adam(env_model.parameters(),ENV_LEARNING_RATE) # Nawid - Optimizer for the env model
    wandb.watch(env_model, log="all")
    env_model_name = 'Env_model'+ '_' + date_time
    early_stopping_env = EarlyStopping_loss(patience=0, verbose=True, wandb=wandb, name=env_model_name)
    for n_iter in range(AGGR_ITER):
        if early_stopping_env.early_stop:
            print('Early stopping')
            break 
        train_model(norm_env_train_data, norm_env_val_data, env_model, BATCH_SIZE, TRAIN_ITER_MODEL,env_optimizer,device,early_stopping_env)

#rew_model = Reward_Oracle()


  1%|          | 1/100 [00:16<27:20, 16.57s/it]

Validation loss decreased/improved for Env_model_17_4_2020-11_0  (100000000000.000000 --> 16.172340).  Saving model ...
epoch: 0 completed



  2%|▏         | 2/100 [00:33<27:04, 16.57s/it]

Validation loss decreased/improved for Env_model_17_4_2020-11_0  (16.172340 --> 15.618386).  Saving model ...
epoch: 1 completed



  3%|▎         | 3/100 [00:48<26:10, 16.19s/it]

...

Validation loss decreased/improved for Env_model_17_4_2020-11_0  (15.279190 --> 15.242322).  Saving model ...
epoch: 5 completed



  7%|▋         | 7/100 [01:50<24:12, 15.61s/it]

Validation loss decreased/improved for Env_model_17_4_2020-11_0  (15.242322 --> 15.228338).  Saving model ...
epoch: 6 completed



  8%|▊         | 8/100 [02:07<24:20, 15.88s/it]

Validation loss decreased/improved for Env_model_17_4_2020-11_0  (15.228338 --> 15.224058).  Saving model ...
epoch: 7 completed
EarlyStopping for Env_model_17_4_2020-11_0 counter: 1 out of 0
Env_model_17_4_2020-11_0 has stopped
Early stopping
Early stopping


Training the reward function and MPC

In [0]:
rew_model = Reward_Oracle()
rand_env_model = NNDynamicsModel(n_actions + feature_size, feature_size).to(device) # Nawid - Need to put both actions as it is a one-hot vector

Controller = multi_model_based_control(ENV_NAME,feature_size,n_actions,possible_positions, rand_env_model, rew_model, NUM_ACTIONS_SEQUENCES, HORIZON_LENGTH)
for n_iter in range(AGGR_ITER):    
    '''
    if early_stopping_env.early_stop:~
        print('Early stopping')
        break 
    '''

    #train_model(norm_env_train_data, norm_env_val_data, env_model, BATCH_SIZE, TRAIN_ITER_MODEL,env_optimizer,device,early_stopping_env)
    

    rl_dataset = Controller.on_policy_collection(STEPS_PER_AGGR)

    '''
    if collect_data:
        env_train_data, env_val_data, rew_train_data, rew_val_data = data_collector.collate_data(rand_dataset,rl_dataset)
        norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data = data_collector.setup_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data)
    '''

  >> R: 160.00, Mean sum:5.00,Survival time: 384,Num examples:384
  >> R: 300.00, Mean sum:4.99,Survival time: 405,Num examples:789
  >> R: 590.00, Mean sum:4.95,Survival time: 801,Num examples:1590
  >> R: 160.00, Mean sum:5.00,Survival time: 243,Num examples:1833
  >> R: 340.00, Mean sum:5.00,Survival time: 417,Num examples:2250
  >> R: 190.00, Mean sum:4.94,Survival time: 365,Num examples:2615
  >> R: 270.00, Mean sum:5.00,Survival time: 393,Num examples:3008
  >> R: 270.00, Mean sum:5.00,Survival time: 351,Num examples:3359
  >> R: 270.00, Mean sum:5.00,Survival time: 357,Num examples:3716
  >> R: 130.00, Mean sum:5.00,Survival time: 183,Num examples:3899
  >> R: 140.00, Mean sum:4.89,Survival time: 177,Num examples:4076
  >> R: 180.00, Mean sum:4.99,Survival time: 303,Num examples:4379
  >> R: 690.00, Mean sum:4.98,Survival time: 561,Num examples:4940
  >> R: 270.00, Mean sum:4.91,Survival time: 333,Num examples:5273
  >> R: 440.00, Mean sum:4.98,Survival time: 615,Num examples:58

UnboundLocalError: ignored

# Old code

In [0]:
# Obtains all the data samples
X_obs, X_act, y_env =env_train_data
X_obs_val, X_act_val, y_env_val = env_val_data
all_X_obs = np.concatenate((X_obs,X_obs_val))

player_positions = all_X_obs[:,8:]
print(player_positions.shape)
enemy1_positions = np.zeros_like(player_positions)
enemy2_positions = np.zeros_like(player_positions)
enemy3_positions = np.zeros_like(player_positions)
enemy4_positions = np.zeros_like(player_positions)

# enemy x positions
enemy1_positions[:,0] = all_X_obs[:,0]
enemy2_positions[:,0] = all_X_obs[:,1]
enemy3_positions[:,0] = all_X_obs[:,2]
enemy4_positions[:,0] = all_X_obs[:,3]

# enemy y positions
enemy1_positions[:,1] = all_X_obs[:,4]
enemy2_positions[:,1] = all_X_obs[:,5]
enemy3_positions[:,1] = all_X_obs[:,6]
enemy4_positions[:,1] = all_X_obs[:,7]
all_agent_positions = np.concatenate((player_positions,enemy1_positions,enemy2_positions,enemy3_positions, enemy4_positions))

(1841002, 2)


In [0]:
unique = np.unique(all_agent_positions, axis=0,return_counts=True)

In [0]:
values = np.sort(unique[1])
print(values[-1600:-1300])


[129 130 132 133 137 137 140 145 147 149 151 153 154 156 159 162 165 168
 168 169 171 172 174 179 179 181 184 184 185 187 188 188 188 189 190 193
 194 197 197 197 202 202 202 203 203 204 205 206 212 212 217 219 219 221
 221 223 223 224 226 226 227 228 228 230 231 235 236 237 237 239 240 243
 245 245 245 246 247 248 248 249 249 250 250 250 252 257 257 259 259 261
 265 265 272 272 273 275 276 276 279 279 281 281 282 283 283 284 284 285
 287 289 290 293 293 293 294 294 295 295 295 297 298 300 300 301 301 302
 302 303 304 306 307 311 312 312 313 314 314 315 316 318 323 325 330 331
 332 332 332 335 335 335 336 337 339 339 343 344 345 347 348 350 350 350
 351 351 353 353 353 358 362 364 364 364 368 369 370 372 372 373 374 375
 375 376 376 378 379 383 383 383 384 386 387 389 390 391 392 392 392 393
 393 393 394 395 397 398 401 402 402 403 404 405 405 405 405 406 407 407
 407 409 412 414 416 416 416 416 418 418 420 426 428 430 431 433 433 435
 436 439 439 440 441 442 442 443 444 444 445 447 44

In [0]:
player_x = X_obs[:,0]
player_y = X_obs[:,1]
max_x = np.amax(player_x)
min_x = np.amin(player_x)
print('min x:', min_x,'max x:',max_x)
max_y = np.amax(player_y)
min_y = np.amin(player_y)
print('min y:', min_y,'max y:',max_y)

unique = np.unique(X_obs, axis=0,return_counts=True)
print(positions.shape)
positions = unique[0]

values = unique[1]

total = values.sum()
print('total:',total)
#print(values[-1])
ordered_values = np.sort(values)
#print(ordered_values[-30]) 
largest_ordered_values = ordered_values[-10:-1] 
print(largest_ordered_values.sum() + ordered_values[-1])

#max_value = np.amax(values)
#print(max_value)

min x: 12 max x: 171
min y: 2 max y: 158


In [0]:
def same(inputs):
    current = inputs[0]
    #print('current current)
    next = inputs[1]
    #print(next)
    if current[0] == next[0] and current[1] == next[1]:
        output = True
    else:
        output = False
    return output

In [0]:
import pandas as pd
df = pd.DataFrame(all_agent_positions,columns=['x', 'y'])
no_dup_df = df.drop_duplicates()
fully_ordered_list = no_dup_df.sort_values(by=['x','y'])
print(fully_ordered_list[0:530])
ordered_data = fully_ordered_list.to_numpy()
#np.save('/content/gdrive/My Drive/MsPacman-data/possible_pacman_positions',ordered_data)
array = np.array([120,80])
print(array.shape)
if array.tolist() in ordered_data.tolist():
    print('present')

         x    y
447369  58   24
318877  58   25
16592   58   26
556     58   50
1816    58   74
54      58   98
1629    58   99
23      58  100
1289    58  101
1428    58  102
1218    58  103
29      58  104
1606    58  105
813     58  106
771     58  107
249     58  108
554     58  109
68      58  110
3388    58  111
98      58  112
7820    58  113
1036    58  114
4557    58  115
1318    58  116
459     58  117
4929    58  118
1521    58  119
337     58  120
832     58  121
536     58  122


In [0]:
next_state = np.zeros((200,8))
next_state[:,4] = 1
next_state[:,5] = 1
#print(next_state)
#print(next_state)
agent_position = np.array((next_state[:,4], next_state[:,5]))
#agent_position = np.array(agent_position)
#print('agent shape',agent_position.shape)
sue_position = np.array((next_state[:,0], next_state[:,1]))
blinky_position = np.array((next_state[:,2], next_state[:,3]))
winky_position = np.array((next_state[:,6], next_state[:,7]))

distance_sue = np.array([np.linalg.norm(agent_position-sue_position, axis=0)])
distance_blinky = np.array([np.linalg.norm(agent_position-blinky_position, axis=0)])
distance_winky = np.array([np.linalg.norm(agent_position- winky_position, axis=0)])
#print(distance_winky.shape)
overall_dist = np.concatenate((distance_sue, distance_blinky, distance_winky),axis = 0)
overall_dist = np.transpose(overall_dist)
print('overall dist',overall_dist.shape)
minElement = np.amin(overall_dist, axis = 1)
print('min element', minElement.shape)

minElement[minElement<1.2] = 0
minElement[minElement>4] = 1
minElement = np.expand_dims(minElement,axis=1)
print(minElement.shape)
'''
minElement = np.amin(distance,axis=0)
print('min element',minElement.shape)
minElement[minElement<1] = 0
minElement[minElement>1] = 1
minElement = np.expand_dims(minElement,axis = 1)
print(minElement.shape)

print('overall_shape',overall_dist.shape)
'''
#enemy_positions = (next_state[:,0], next_state[:,1]),(next_state[:,2], next_state[:,3]),(next_state[:6], next_state[:,7])
#enemy_positions = np.array(enemy_positions)
#print('enemy positions',enemy_positions.shape)

'''
distance = np.array([np.linalg.norm(agent_position-x,axis = 0) for x in enemy_positions])
print('distance',distance.shape)
minElement = np.amin(distance,axis=0)
print('min element',minElement.shape)
minElement[minElement<1] = 0
minElement[minElement>1] = 1
minElement = np.expand_dims(minElement,axis = 1)
print(minElement.shape)
'''     

'''
for x in enemy_positions:
    dist = agent_position - x
    distance = np.array(np.linalg.norm(dist,axis = 0))
    print(distance.shape)
'''
    #print(dist.shape)
#print(enemy_positions.shape)

#distance = np.array(np.linalg.norm(agent_position -enemy_positions,axis = 1))
#print(distance)

overall dist (200, 3)
min element (200,)
(200, 1)


'\nfor x in enemy_positions:\n    dist = agent_position - x\n    distance = np.array(np.linalg.norm(dist,axis = 0))\n    print(distance.shape)\n'

In [0]:
c = np.array([[ 1, 2, 3],[-1, 1, 4]])
print(c.shape)

(2, 3)


In [0]:
next_state = np.zeros((3,10))
#print(x.shape)
next_state[:,0:7] = 1


agent_position = [(next_state[:,8], next_state[:,9])]
agent_position = np.array(agent_position)

enemy_positions = [(next_state[:,0],next_state[:,4]),(next_state[:,1],next_state[:,5]),(next_state[:,2],next_state[:,6]),(next_state[:,3],next_state[:,7])]
enemy_positions = np.array(enemy_positions)
distance = [np.linalg.norm(agent_position[0]-x,axis = 1) for x in enemy_positions]
min_distance = np.amin(distance)

if min_distance < 1:
    reward = 0
else:
    reward = 1

#np.linalg.norm(axis =0)
print(distance)   

[array([1.73205081, 1.73205081]), array([1.73205081, 1.73205081]), array([1.73205081, 1.73205081]), array([1.73205081, 0.        ])]


In [0]:
for var_name in env_optimizer.state_dict():
    print(var_name)
    print(env_optimizer.state_dict()['param_groups'][0]['lr'])
    #print(var_name, "\t", env_optimizer.state_dict()[var_name])

In [0]:
import numpy as np
agent_position = [(0,1)]
agent_position = np.array(agent_position)
centerList = [(1,2),(54, 2991), (1717, 2989), (1683, 2991), (1604, 2991), (114, 2991), (919,222), (930,233)]
centerList = np.array(centerList)

#print(centerList[0])
#print(centerList[0])
values = np.array([np.linalg.norm(agent_position[0]-x) for x in centerList])

#print(centerList.shape)
print(values)
#print(new_values)
agent = np.array([1,2])
print(agent.shape)
print(agent_position.shape)
minElement = np.amin(values)
 
print('Minimum element from Numpy Array : ', minElement)
def getDistance(p0,p1):
    return np.linalg.norm(p0-p1)

#getDistance(centerList[0], centerList[1])

[1.41421356e+00 2.99048759e+03 3.44619109e+03 3.43112066e+03
 3.39306882e+03 2.99217245e+03 9.45199450e+02 9.58500913e+02]
(2,)
(1, 2)
Minimum element from Numpy Array :  1.4142135623730951


In [0]:
config = wandb.config
config.batch_size = BATCH_SIZE          
config.horizon_length = HORIZON_LENGTH
config.num_action_seq = NUM_ACTIONS_SEQUENCES
config.train_model_iter = TRAIN_ITER_MODEL 
config.num_rand_trajectories = NUM_RAND_TRAJECTORIES
config.aggr_iter = AGGR_ITER
config.steps_per_aggr = STEPS_PER_AGGR

config.env_lr = ENV_LEARNING_RATE


config.rew_oracle = rew_oracle
     
config.no_actions = n_actions
config.load_data = load_data
config.collect_data = collect_data
config.single_life = single_life
config.random_seed = random_seed

if load_data:
    config.loaded_trajectories = loaded_trajectories

In [0]:
counter = 0
#current = rand_dataset[1,0]
#next = rand_dataset[2,1]

#if current[0] == next[0] and current[1] == next[1]:
#    print('same')
#print(current)
#print(next)

def same(inputs):
    current = inputs[0]
    #print('current current)
    next = inputs[1]
    #print(next)
    if current[0] == next[0] and current[1] == next[1]:
        output = True
    else:
        output = False
    return output



print('rand',rand_dataset[0])
some_list = np.array([x for x in rand_dataset if not same(x)])
print('some list',some_list.shape)
'''
for i in range(len(rand_dataset)):
    current = rand_dataset[i,0]
    next = rand_dataset[i,1]
    if current[0] == next[0] and current[1] == next[1]:
        counter += 1 

print(counter)
'''

rand [array([88, 98], dtype=uint8) array([85, 98], dtype=uint8) 0.0 False
 array([0, 1, 0, 0, 0], dtype=uint8)]
some list (301909, 5)


'\nfor i in range(len(rand_dataset)):\n    current = rand_dataset[i,0]\n    next = rand_dataset[i,1]\n    if current[0] == next[0] and current[1] == next[1]:\n        counter += 1 \n\nprint(counter)\n'

In [0]:

#unique = np.unique(all_agent_positions, axis=0)
#ordered_values = np.sort(unique)
#ordered_values.shape

#all_agent_positions_x = all_agent_positions[:,1]
#all_agent_positions_x =

#new_array = [tuple(row) for row in all_agent_positions]
#unique_tuples = np.unique(new_array)

'''
max_x = np.amax(all_agent_positions_x)
min_x = np.amin(all_agent_positions_x)
print('min x:', min_x,'max x:',max_x)

a = np.array([[1, 1], [1, 0 ],[2,0],[3,0]])
print(a.shape)
unique_prac = np.unique(a,axis=0)
print(unique_prac)

list_of_values = []
list_of_values.append([1,2])
list_of_values.append([2,3])
final_list = np.array(list_of_values)
print(final_list)


def manual_sorting(dataset):
    final_list = []
    upper_val = len(dataset)
    x = 11
    while x < 172:
        y = 1
        while y < 159:
            if dataset[i] == [x,y]:
                final_list.append([x,y])
                y +=1
                i = 0
            else:
                i +=1
            
            if i == upper_val - 1:
                y += 1
                i = 0
            # if all y values have been seen, increase the x value
            if y == 158:
                x +=1
                
'''